<a href="https://colab.research.google.com/github/johnyamarun/run-monitor/blob/main/FIT_FORM_ANALYZER_release_v1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

セル1: 環境構築とマウント

In [ ]:
!pip install fitparse pandas

import os
import glob
import datetime
import pandas as pd
from fitparse import FitFile
from google.colab import drive
from IPython.display import display, HTML
import json

# Googleドライブのマウント
drive.mount('/content/drive')

セル2: ユーザー設定（Colabフォーム）

In [ ]:
#@title ランナー基本情報とレポート設定
#@markdown 以下の項目を入力してください。最大心拍数が不明な場合は `0` を入力してください。

USER_AGE = 35 #@param {type:"integer"}
USER_MAX_HR = 0 #@param {type:"integer"}
#@markdown ---
#@markdown **アプリ等で把握しているLT値（任意・不明なら0または空欄）**
USER_LT_HR = 0 #@param {type:"integer"}
USER_LT_PACE = "0" #@param {type:"string"}
#@markdown ---
REPORT_DAYS = 7 #@param {type:"integer"}

# 最大心拍数の補完
if USER_MAX_HR == 0:
    CALCULATED_MAX_HR = int(208 - 0.7 * USER_AGE)
    print(f"最大心拍数が未入力のため、年齢から推計しました: {CALCULATED_MAX_HR} bpm")
else:
    CALCULATED_MAX_HR = USER_MAX_HR
    print(f"設定された最大心拍数を使用します: {CALCULATED_MAX_HR} bpm")

FIT_DIR = '/content/drive/MyDrive/FIT_Data'

セル3: データ抽出ロジックとサマリー表示

In [ ]:
def extract_real_fit_summary(file_path):
    try:
        fitfile = FitFile(file_path)
        summary = {
            "file_name": os.path.basename(file_path),
            "timestamp": None,
            "date": None,
            "distance_km": 0.0,
            "avg_pace": "0:00",
            "max_pace": "0:00",
            "max_speed": 0.0,
            "avg_hr": 0,
            "max_hr": 0,
            "avg_cadence": 0,
            "avg_stride": 0.0,
            "avg_gct": 0.0,
            "avg_vertical_oscillation": 0.0,
            "has_pod": False,
            "has_ext_hr": False,
            "zones": {"Z1": 0.0, "Z2": 0.0, "Z3": 0.0, "Z4": 0.0, "Z5": 0.0}
        }

        hr_list = [] # 毎秒の心拍データを格納するリスト

        for record in fitfile.get_messages():
            if record.name == 'session':
                for data in record:
                    if data.name == 'start_time' and data.value:
                        summary['timestamp'] = data.value
                        summary['date'] = data.value.strftime("%Y-%m-%d")
                    elif data.name == 'total_distance' and data.value:
                        summary['distance_km'] = round(data.value / 1000, 2)
                    elif data.name == 'avg_speed' and data.value > 0:
                        pace_sec = 1000 / data.value
                        summary['avg_pace'] = f"{int(pace_sec // 60)}:{int(pace_sec % 60):02d}"
                    elif data.name == 'max_speed' and data.value > 0:
                        summary['max_speed'] = data.value
                        m_pace_sec = 1000 / data.value
                        summary['max_pace'] = f"{int(m_pace_sec // 60)}:{int(m_pace_sec % 60):02d}"
                    elif data.name == 'avg_heart_rate' and data.value:
                        summary['avg_hr'] = data.value
                    elif data.name == 'max_heart_rate' and data.value:
                        summary['max_hr'] = data.value
                    elif data.name in ['avg_cadence', 'avg_running_cadence'] and data.value:
                        summary['avg_cadence'] = int(data.value * 2) if data.value < 130 else int(data.value)
                    elif data.name in ['avg_step_length', 'avg_stride_length'] and data.value:
                        summary['avg_stride'] = round(data.value / 10, 1)
                    elif data.name == 'avg_stance_time' and data.value:
                        summary['avg_gct'] = round(data.value, 1)
                    elif data.name == 'avg_vertical_oscillation' and data.value:
                        summary['avg_vertical_oscillation'] = round(data.value / 10, 1)
            elif record.name == 'device_info':
                for data in record:
                    if data.name == 'device_type' and data.value == 120:
                        summary['has_ext_hr'] = True
            elif record.name == 'record':
                for data in record:
                    if data.name in ['stance_time_balance', 'vertical_oscillation', 'leg_spring_stiffness']:
                        summary['has_pod'] = True
                    # 毎秒の心拍数を抽出
                    if data.name == 'heart_rate' and data.value:
                        hr_list.append(data.value)

        # 心拍ゾーンの滞在割合（%）を計算
        if hr_list and CALCULATED_MAX_HR > 0:
            total_hr = len(hr_list)
            z_counts = {"Z1": 0, "Z2": 0, "Z3": 0, "Z4": 0, "Z5": 0}
            for hr in hr_list:
                pct = hr / CALCULATED_MAX_HR
                if pct >= 0.90: z_counts["Z5"] += 1
                elif pct >= 0.80: z_counts["Z4"] += 1
                elif pct >= 0.70: z_counts["Z3"] += 1
                elif pct >= 0.60: z_counts["Z2"] += 1
                elif pct >= 0.50: z_counts["Z1"] += 1
            summary['zones'] = {k: round((v / total_hr) * 100, 1) for k, v in z_counts.items()}

        if not summary['timestamp']:
            summary['timestamp'] = datetime.datetime.fromtimestamp(os.path.getmtime(file_path))
            summary['date'] = summary['timestamp'].strftime("%Y-%m-%d")

        return summary
    except Exception as e:
        print(f"読み込みエラー: {os.path.basename(file_path)} - {e}")
        return None

# ファイル一覧の取得と日付順ソート
fit_files = glob.glob(os.path.join(FIT_DIR, '*.fit'))
all_data = []
daily_data_list = []
period_data_list = []

if not fit_files:
    print(f"【警告】{FIT_DIR} にFITファイルが見つかりません。")
else:
    for f in fit_files:
        d = extract_real_fit_summary(f)
        if d:
            all_data.append(d)

    # FITファイル内の実際の走行日時で降順ソート（最新が先頭）
    all_data.sort(key=lambda x: x['timestamp'], reverse=True)

    if all_data:
        latest_date = all_data[0]['date']
        daily_data_list = [d for d in all_data if d['date'] == latest_date]
        daily_data_list.sort(key=lambda x: x['timestamp'])

        limit_date = all_data[0]['timestamp'] - datetime.timedelta(days=REPORT_DAYS)
        period_data_list = [d for d in all_data if d['timestamp'] >= limit_date]

        print(f"=== {REPORT_DAYS}日間のデータ読み取り完了（最新日: {latest_date} / この日のセッション数: {len(daily_data_list)}件） ===")

セル4: プロンプト生成とコピーボタン

In [ ]:
def estimate_workout_type(d, max_hr_user):
    """生理学的指標に基づく練習タイプ推定"""
    if max_hr_user <= 0 or d['avg_hr'] == 0: return "不明"

    avg_hr_pct = d['avg_hr'] / max_hr_user
    max_hr_pct = d['max_hr'] / max_hr_user
    hr_drift = d['max_hr'] - d['avg_hr']

    max_pace_sec = 1000 / d['max_speed'] if d.get('max_speed', 0) > 0 else 9999
    is_high_speed = max_pace_sec < 240

    # インターバル判定
    if (max_hr_pct >= 0.90 and hr_drift >= 20) or (max_hr_pct >= 0.88 and hr_drift >= 25 and is_high_speed):
        return "VO2max / インターバル"

    if avg_hr_pct > 0.88: return "レース / タイムトライアル"
    if 0.82 <= avg_hr_pct <= 0.88: return "LT / 閾値走"
    if d['distance_km'] >= 25.0 and avg_hr_pct < 0.80: return "ロング走"
    if avg_hr_pct < 0.75: return "回復走 / イージージョグ"
    return "ジョグ / モデレート"

def create_copy_button(text, button_label):
    text_json = json.dumps(text)
    html_code = f"""
    <div style="margin-top: 10px; margin-bottom: 30px;">
        <button onclick='navigator.clipboard.writeText({text_json}).then(()=>alert("Copied."))'
                style="padding: 8px 16px; font-size: 14px; font-weight: bold; color: white; background-color: #4285F4; border: none; border-radius: 4px; cursor: pointer;">
            {button_label}
        </button>
    </div>
    """
    display(HTML(html_code))

def format_metric(val, unit):
    """0.0などの欠損値を「計測なし」に変換するフォーマッター"""
    if val == 0.0 or val == 0:
        return "計測なし"
    return f"{val}{unit}"

def generate_llm_prompt(data_list, is_weekly=False):
    uses_ext_hr = any(d['has_ext_hr'] for d in data_list)
    uses_foot_pod = any(d['has_pod'] for d in data_list)

    sensor_note = "※胸部/上腕心拍センサー使用：心拍精度高" if uses_ext_hr else "※手首計測：心拍ラグ・ケイデンスロックの可能性あり"
    pod_note = "※フットポッド使用：接地時間・上下動・左右バランスの精度極めて高" if uses_foot_pod else "※ウォッチ単体：バイオメカニクス値は傾向として捉えてください"

    lt_context = ""
    if USER_LT_HR > 0 and USER_LT_PACE:
        lt_context = f"- ユーザー申告LT値: 心拍 {USER_LT_HR} bpm / ペース {USER_LT_PACE} /km\n"

    base_prompt = f"""あなたは高度な運動生理学とバイオメカニクスに精通したエリートランニングコーチです。
以下のデータを詳細に分析し、ランニングサイエンスに基づいた具体的なフィードバックを提供してください。

【コンテキスト（前提条件）】
- ランナー設定: {USER_AGE}歳、最大心拍数 {CALCULATED_MAX_HR} bpm
{lt_context}- 計測環境: {sensor_note}、{pod_note}

【データ解釈における重要ルール（必ず遵守すること）】
1. インターバル練習の平均値: VO2max/インターバル練習において、平均ピッチや平均ストライドが低く算出されている場合、それは「レスト区間の歩行や静止」が含まれているためです。これをフォームの悪化と誤認して指摘しないでください。
2. ジョグの接地時間（GCT）: 回復走やイージージョグにおいて接地時間が長くなるのは、ペース低下に伴う自然な力学的現象です。これを問題視せず、GCTや上下動の評価は主に「LT走」や「レースペース」などの高強度セッションのデータで行ってください。
3. データ欠損: 「計測なし」となっている項目は、室内ラン等によるデバイスの仕様によるものです。データがないことを指摘・批判する必要はありません。
"""

    if not is_weekly:
        total_distance = sum(d['distance_km'] for d in data_list)
        content = f"\n【今日のトレーニング詳細（1日合計: {round(total_distance, 2)} km）】\n"

        for i, d in enumerate(data_list):
            w_type = estimate_workout_type(d, CALCULATED_MAX_HR)
            start_time_str = d['timestamp'].strftime('%H:%M') if d['timestamp'] else '不明'
            z = d['zones']

            # フォーマッターを通す
            f_cadence = format_metric(d['avg_cadence'], "spm")
            f_stride = format_metric(d['avg_stride'], "cm")
            f_gct = format_metric(d['avg_gct'], "ms")
            f_vo = format_metric(d['avg_vertical_oscillation'], "cm")

            content += f"◆ セッション {i+1} [{w_type}] ({start_time_str} 開始)\n"
            content += f"- 距離: {d['distance_km']} km | ペース: 平均 {d['avg_pace']} /km (最速 {d['max_pace']} /km)\n"
            content += f"- 心拍: 平均 {d['avg_hr']} bpm (最大 {d['max_hr']} bpm / {int((d['max_hr']/CALCULATED_MAX_HR)*100)}%)\n"
            content += f"- ゾーン滞在割合: Z1({z['Z1']}%) Z2({z['Z2']}%) Z3({z['Z3']}%) Z4({z['Z4']}%) Z5({z['Z5']}%)\n"
            content += f"- ピッチ: {f_cadence} | ストライド: {f_stride}\n"
            content += f"- 接地時間: {f_gct} | 上下動: {f_vo}\n\n"

        content += """【コーチへの指示】
1. 1日のセッション構成と各セッションのゾーン滞在割合から、目的に沿ったトレーニングができているか（メリハリや狙った強度の達成度）を評価してください。
2. バイオメカニクス（力学）データの評価は、重要ルールに従い、適切なセッションでのみ行ってください。
3. 明日以降へのアドバイスと、理学療法的な補強運動を1つ提案してください。
「鬼コーチ」「天使コーチ」の両モードで出力してください。
"""
    else:
        content = f"\n【直近{REPORT_DAYS}日間のトレーニングサマリー】\n"
        for d in data_list:
            w = estimate_workout_type(d, CALCULATED_MAX_HR)
            z = d['zones']

            f_cadence = format_metric(d['avg_cadence'], "spm")
            f_stride = format_metric(d['avg_stride'], "cm")
            f_gct = format_metric(d['avg_gct'], "ms")
            f_vo = format_metric(d['avg_vertical_oscillation'], "cm")

            content += f"- {d['date']} [{w}]: {d['distance_km']}km, ペース平均{d['avg_pace']}(最速{d['max_pace']}), HR(avg{d['avg_hr']}/max{d['max_hr']}), ゾーン[Z1:{z['Z1']}% Z2:{z['Z2']}% Z3:{z['Z3']}% Z4:{z['Z4']}% Z5:{z['Z5']}%], {f_cadence}, {f_stride}, VO {f_vo}, GCT {f_gct}\n"

        content += """
【コーチへの指示】
1. 練習のバリエーションとゾーン滞在割合から、期間全体の強度のバランス（ポラライズドトレーニング等の目的）を評価してください。
2. 疲労の蓄積度や「フォームの崩れ」の予兆は、重要ルールを遵守し、比較可能な高強度セッションの推移から読み取ってください。
3. 今後の能力向上のための「戦略的処方箋」を提示してください。
"""

    return base_prompt + content

# 実行・表示
if all_data and daily_data_list:
    print("="*50 + "\n【日々のレポート（デイリー分析）】\n" + "="*50)
    p_daily = generate_llm_prompt(daily_data_list, False)
    print(p_daily)
    create_copy_button(p_daily, "デイリー用プロンプトをコピー")

    if len(period_data_list) > 1:
        print("="*50 + f"\n【振り返りレポート（直近{REPORT_DAYS}日間）】\n" + "="*50)
        p_period = generate_llm_prompt(period_data_list, True)
        print(p_period)
        create_copy_button(p_period, "振り返り用プロンプトをコピー")